In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time
import random

import Init_NetRate as Init
import cvxpy as cp
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx
import NetRate_ADMM_fcts as ADMM
import NetRate_SG_fcts as NetSG

import Init_NetInf
import Greedy_NetInf as Greed

In [2]:
def Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat,lower_bound):
    correct = 0
    nb_edges = 0
    top_acc = 0
    bot_acc = 0
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if A_hat[i,j] >lower_bound:
                nb_edges +=1
                bot_acc +=1
                if (i,j) in G_true.edges():
                    correct +=1
                    bot_acc+=1
                else :
                    top_acc+=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc+=1
                
    precision = correct/nb_edges
    recall = correct/G_true.number_of_edges()
    accuracy = 1-(top_acc/bot_acc)
    mse = 0
    nb_correcte_edges=0
    for edge in G_true.edges():
            try:
                true_alpha = G_true.edges[edge]["weight"][0]
            except TypeError:
                true_alpha = G_true.edges[edge]["weight"]
            approx_alpha = A_hat[edge[0],edge[1]]
            mse += pow((true_alpha-approx_alpha),2) # mean square error
    try :
        mse = mse/G_true.number_of_edges()
    except ZeroDivisionError:
        print ("There is no correct edge")
    return (precision,recall,accuracy,mse)

def Global_Precision_Recall_Accuracy_NetInf(G_true,G_max):
    correct = 0
    top_acc = 0
    bot_acc = 0
    nb_edges_G_max = G_max.number_of_edges()
    nb_edges_G_true = G_true.number_of_edges()
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if (i,j) in G_max.edges() :
                bot_acc +=1
                if(i,j) in G_true.edges() :
                    bot_acc +=1
                    correct+=1
                else :
                    top_acc +=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc +=1
    precision = correct/nb_edges_G_max
    recall = correct/nb_edges_G_true
    accuracy = 1-top_acc/bot_acc
    return(precision,recall,accuracy)

def Write_info(file_path,time,algo_infos):
    precision,recall,accuracy,mse = algo_infos
    f = open(file_path,"w")
    f.write("General Infos\n")
    f.write("time needed : "+str(time)+" s\n")
    f.write("Accuracy of the algo : " +str(accuracy)+"\n")
    f.write("precision of the algo : "+str(precision)+"\n")
    f.write("recall of the algo : "+str(recall)+"\n")
    f.write("mean square error is : "+str(mse)+"\n")
    f.close()

# Parameters

In [21]:
'''
Global parameters for the cascade model and for all solving methods.
'''
window = 10
beta = 1 # used for the construction of the cascades, probability of the edge to transmit the infection
alpha_min = 0.0005
alpha_max = 10 #bound alpha from above
alpha_init = 0.01 #initial value of the infectious rates
tol = 0.05 # value from which we decide that there exist an edge
more_infos_Netrate = False # If set to true computes the time,accuracy and mean square error at each iteration (solves down the overall computartion)
# can also be modified for each solver.

solvers = ["CVX","SGD","ADMM"] #List containing the solver that should be used. "NetInf","CVX","SGD","ADMM"
'''
NetRate with CVX
'''
more_infos_CVX = more_infos_Netrate

'''
NetRate with SGD parameters
'''
gamma_SGD = 0.0005
K = 15000 #number of itterations of SGD
more_infos_SGD = more_infos_Netrate

SGD_param = (alpha_init,alpha_max,K,gamma_SGD,alpha_min,window,tol,more_infos_SGD)

'''
Netrate with ADMM
'''
iter_ADMM = 50 #number of itteration for 1 node in the ADMM method. This is a parameter to tune
iter_GD_ADMM = 1000 #number of iterations of the gradient descent
gamma_ADMM = 0.000005 # Learning rate of the GD for alpha (maybe too small)
u = 4 # used for the gradient descent of rho and as a penalizer and the constrain
more_infos_ADMM = more_infos_Netrate 
param_ADMM = (alpha_init,alpha_max,window,iter_GD_ADMM,iter_ADMM,gamma_ADMM,u,alpha_min,tol,more_infos_ADMM)


'''
NetInf paramters
'''
EPS = 1e-64 #zero machine
ALPHA = 1.0 #Incubation parameter (for exp and power law)
MODEL = 0 # 0 = exp law, 1 = power law (power law is not  implemented yet) When constructing the underlying network specify that we use the exponential law
MAX = sys.float_info.max #Max value of a float in python
#MIN = sys.float_info.min #Min value of a float in python
MIN = -MAX
#(works only if groundtruth is available)
#When set to True (especially boundOn) it slow down greatly the computation
compare_groud_truth = False # If set to True outputs some aditional information (precision and recall of the algo)
boundOn = False

greedy_global_param = (ALPHA,MODEL,MAX,MIN,EPS,compare_groud_truth,boundOn)


# Script solving the optimization problem and saving the results

In [11]:
graph_list = [(100,100,100)] # list of the graph we want to generate. Each graph is represented by a triple (vertex,edges,nb_cascades)
for i in range(0,len(graph_list)) :
    (vertex,edge,cascades) = graph_list[i]
    
    '''
    Creation of folder where the infos will be saved
    '''
    folder_name = "Simulation_"+str(vertex)+"V_"+str(edge)+"E_"+str(cascades)+"C"
    try:
        os.mkdir(folder_name)
    except FileExistsError :
        print("folder exists already")
    folder_path = "./"+folder_name    
    graph_file_name = "Graph_true_"+str(vertex)+"V_"+str(edge)+"E.txt"
    cascade_file_name = "Cascades_"+str(vertex)+"V_"+str(edge)+"E.txt"
    graph_path = os.path.join(folder_path+"/"+graph_file_name)
    cascade_path = os.path.join(folder_path+"/"+cascade_file_name)
    
    '''
    Generation of ground truh and cascades. Save them in the above defined folder.
    '''
    G_true = Gen.Generate_random_graph(vertex,edge) # Generates the underlying network
    Cascades = Gen.Generate_all_cascades(G_true,-cascades,window,MODEL,beta) # generates the cascades
    Gen.Save_cascade_to_file(cascade_path,Cascades,G_true)
    Gen.Save_graph_to_file(graph_path,G_true)
    
    '''
    Initialization for NetRate
    '''
    G_star,C_dic = Init.Init(cascade_path)
    N = G_true.number_of_nodes()
    
    '''
    Initialization for NetRate
    '''
    if "NetInf" in solvers :
        print("**********NetInf**********")
        G_star_NF,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF = Init_NetInf.Init(cascade_path,EPS,MAX)
        nb_max_edge = int(1*G_true.number_of_edges()) #fix a number of edges we want to recover 
        ground_truth = G_true
        t_s = time.time()
        G_max,precision,recall,edge_info = Greed.GreedyOpt(nb_max_edge,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF,G_star_NF,True,greedy_global_param)
        t_f = time.time()
        algo_info = Global_Precision_Recall_Accuracy_NetInf(G_true,G_max)
        algo_info = algo_info+(-15,)
        time_algo = t_f-t_s
        general_info_path = os.path.join(folder_path+"/General_Infos_NetInf.txt")
        Write_info(general_info_path,time_algo,algo_info)
        print(time_algo)
    
    if "CVX" in solvers :
        print("**********NetRate_CVX**********")
        A_list,nb_c_per_node = cvx.Create_matrices(N,C_dic,window)
        t_s = time.time()
        A_hat_CVX,final_obj_value,acc_mse_time_CVX = cvx.Infer_Network_edges(N,A_list,nb_c_per_node,C_dic,more_infos_CVX)
        t_f = time.time()
        time_algo_CVX=t_f-t_s
        algo_infos_CVX = Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_CVX,tol)
        np.save(os.path.join(folder_path+"/"+"alpha_CVX"),A_hat_CVX)
        general_info_path_CVX = os.path.join(folder_path+"/General_Infos_CVX.txt")
        Write_info(general_info_path_CVX,time_algo_CVX,algo_infos_CVX)
        print(time_algo_CVX)
    
    if "SGD" in solvers :
        print("**********NetRate_SGD**********")
        t_s = time.time()
        A_hat_SG,acc_mse_time_SGD = NetSG.Netrate_SGD(G_true,C_dic,N,SGD_param)
        t_f = time.time()
        time_algo_SGD = t_f-t_s      
        algo_infos_SGD = Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_SG,tol)
        np.save(os.path.join(folder_path+"/"+"alpha_SGD"),A_hat_SG)
        general_info_path_SGD = os.path.join(folder_path+"/General_Infos_SGD.txt")
        Write_info(general_info_path_SGD,time_algo_SGD,algo_infos_SGD)
        print(time_algo_SGD)
    
    if "ADMM" in solvers :
        print("**********NetRate_ADMM**********")
        t_s = time.time()
        A_hat_ADMM,obj_per_iter_per_node_ADMM,acc_mse_time_ADMM = ADMM.NetRate_ADMM(G_true,C_dic,N,param_ADMM)
        t_f = time.time()
        time_algo_ADMM = t_f-t_s
        algo_infos_ADMM = Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)
        np.save(os.path.join(folder_path+"/alpha_ADMM"),A_hat_ADMM)
        general_info_path_ADMM = os.path.join(folder_path+"/General_Infos_ADMM.txt")
        Write_info(general_info_path_ADMM,time_algo_ADMM,algo_infos_ADMM)
        print(time_algo_ADMM)
        
        
    '''
    Plots for each graph solved. The plots are stored in the corresponding folder
    '''    
    time_ADMM = []
    acc_ADMM = []
    mse_ADMM = []
    time_SGD = []
    acc_SGD = []
    mse_SGD = []
    time_CVX = []
    acc_CVX = []
    mse_CVX = []
    
    if more_infos_ADMM and "ADMM" in solvers :
        for bla in acc_mse_time_ADMM :
            acc_ADMM.append(bla[0])
            mse_ADMM.append(bla[1])
            time_ADMM.append(bla[2])

        time_cum_ADMM = np.cumsum(time_ADMM)

    if more_infos_SGD and "SGD" in solvers :        
        for plop in acc_mse_time_SGD :
            acc_SGD.append(plop[0])
            mse_SGD.append(plop[1])
            time_SGD.append(plop[2])

        time_cum_SGD = np.cumsum(time_SGD)

    if more_infos_CVX and "CVX" in solvers :        
        for plop in acc_mse_time_CVX:
            acc_CVX.append(plop[0])
            mse_CVX.append(plop[1])
            time_CVX.append(plop[2])
        time_cum_CVX = np.cumsum(time_CVX)

    if more_infos_CVX or more_infos_ADMM or more_infos_ADMM :
        
        plt.figure(1)
        plt.plot(time_cum_ADMM,acc_ADMM,label = 'ADMM')
        plt.plot(time_cum_SGD,acc_SGD,label = "SGD")
        plt.plot(time_cum_CVX,acc_CVX,label = "CVX")
        plt.xlabel("running time (s)")
        plt.ylabel("Accuracy")
        plt.legend(loc = 'lower right')
        plt.title("G=(%i,%i) with |C| = %i" %(vertex,edge,cascades))
        plt.savefig(os.path.join(folder_path+"/ACC_TIME"))
        plt.show()


        plt.figure(2)
        plt.plot(time_cum_ADMM,mse_ADMM,label = "ADMM")
        plt.plot(time_cum_SGD,mse_SGD,label = "SGD")
        plt.plot(time_cum_CVX,mse_CVX,label = "CVX")
        plt.ylabel("Mean square error (MSE)")
        plt.xlabel("running time (s)")
        plt.title("G=(%i,%i) with |C| = %i" %(vertex,edge,cascades))
        plt.legend(loc = "best")
        plt.savefig(os.path.join(folder_path+"/MSE_TIME"))
        plt.show()

All nodes were read
**********NetRate_CVX**********
For node  0
For node  1
For node  2
For node  3
For node  4
For node  5
For node  6
For node  7
For node  8
For node  9
For node  10
For node  11
For node  12
For node  13
For node  14
For node  15
For node  16
For node  17
For node  18
For node  19
For node  20
For node  21
For node  22
For node  23
For node  24
For node  25
For node  26
For node  27
For node  28
For node  29
For node  30
For node  31
For node  32
For node  33
For node  34
For node  35
For node  36
For node  37
For node  38
For node  39
For node  40
For node  41
For node  42
For node  43
For node  44
For node  45
For node  46
For node  47
For node  48
For node  49
For node  50
For node  51
For node  52
For node  53
For node  54
For node  55
For node  56
For node  57
For node  58
For node  59
For node  60
For node  61
For node  62
For node  63
For node  64
For node  65
For node  66
For node  67
For node  68
For node  69
For node  70
For node  71
For node  72
For node 

In [22]:
time_ADMM = []
acc_ADMM = []
mse_ADMM = []
time_SGD = []
acc_SGD = []
mse_SGD = []
time_CVX = []
acc_CVX = []
mse_CVX = []
if more_infos_ADMM and "ADMM" in solvers :
    for bla in acc_mse_time_ADMM :
        acc_ADMM.append(bla[0])
        mse_ADMM.append(bla[1])
        time_ADMM.append(bla[2])
        
    time_cum_ADMM = np.cumsum(time_ADMM)

if more_infos_SGD and "SGD" in solvers :        
    for plop in acc_mse_time_SGD :
        acc_SGD.append(plop[0])
        mse_SGD.append(plop[1])
        time_SGD.append(plop[2])
        
    time_cum_SGD = np.cumsum(time_SGD)

if more_infos_CVX and "CVX" in solvers :        
    for plop in acc_mse_time_CVX:
        acc_CVX.append(plop[0])
        mse_CVX.append(plop[1])
        time_CVX.append(plop[2])
    time_cum_CVX = np.cumsum(time_CVX)
        
if more_infos_CVX or more_infos_ADMM or more_infos_ADMM :
    plt.figure(1)
    plt.plot(time_cum_ADMM,acc_ADMM,label = 'ADMM')
    plt.plot(time_cum_SGD,acc_SGD,label = "SGD")
    plt.plot(time_cum_CVX,acc_CVX,label = "CVX")
    plt.xlabel("running time (s)")
    plt.ylabel("Accuracy")
    plt.legend(loc = 'lower right')
    plt.title("G=(%i,%i) with |C| = %i" %(vertex,edge,cascades))
    plt.savefig(os.path.join(folder_path+"/ACC_TIME"))
    plt.show()


    plt.figure(2)
    plt.plot(time_cum_ADMM,mse_ADMM,label = "ADMM")
    plt.plot(time_cum_SGD,mse_SGD,label = "SGD")
    plt.plot(time_cum_CVX,mse_CVX,label = "CVX")
    plt.ylabel("Mean square error (MSE)")
    plt.xlabel("running time (s)")
    plt.title("G=(%i,%i) with |C| = %i" %(vertex,edge,cascades))
    plt.legend(loc = "best")
    plt.savefig(os.path.join(folder_path+"/MSE_TIME"))
    plt.show()

# Individual functions

In [ ]:
# G_true = Init.Load_ground_truth(network_file_name)
# G_star, DAG_C_dic = Init.Init(cascade_file_name)
# N = G_true.number_of_nodes()
cascade_file = "Cascade_gen.txt"
graph_true_file ="test.txt"
G_true = Gen.Generate_random_graph(100,200)
Cascades = Gen.Generate_all_cascades(G_true,-100,window,MODEL,beta)
Gen.Save_cascade_to_file(cascade_file,Cascades,G_true)
Gen.Save_graph_to_file(graph_true_file,G_true)
G_star,C_dic = Init.Init(cascade_file)
N = G_true.number_of_nodes()


# G_star_NF,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF = Init_NetInf.Init(cascade_file,EPS,MAX)

In [ ]:
nb_max_edge = int(1*G_true.number_of_edges()) #fix a number of edges we want to recover (here 90%)
ground_truth = G_true
t_s = time.time()
G_max,precision,recall,edge_info = Greed.GreedyOpt(nb_max_edge,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF,G_star_NF,True,greedy_global_param)
t_f = time.time()
Global_Precision_Recall_Accuracy_NetInf(G_true,G_max)
print ("Netinf took : ",t_f-t_s)

In [ ]:
A_list,nb_c_per_node = cvx.Create_matrices(N,C_dic,window)
t_s = time.time()
alpha_mat,final_obj_value,acc_mse_time_CVX = cvx.Infer_Network_edges(N,A_list,nb_c_per_node,C_dic,more_infos_CVX)
t_f = time.time()
print("NetRate CVX took : ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,alpha_mat,0.05)
np.save("alpha_CVX",alpha_mat)

In [ ]:
t_s = time.time()
A_hat_SG,acc_mse_time_SGD = NetSG.Netrate_SGD(G_true,C_dic,N,SGD_param)
t_f = time.time()
print("NetRate SG took ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_SG,0.05)
np.save("alpha_SGD",A_hat_SG)

In [ ]:
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_SG,0.05)

In [ ]:
t_s = time.time()
A_hat_ADMM,obj_per_iter_per_node_ADMM,acc_mse_time_ADMM = ADMM.NetRate_ADMM(G_true,C_dic,N,param_ADMM)
t_f = time.time()
print("ADMM took ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)
np.save("alpha_ADMM",A_hat_ADMM)

In [ ]:
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)

In [ ]:
count = 0
for i in range(0,N):
    for j in range(0,N):
        if A_hat_ADMM[i,j]>1 :
            count +=1
print(count)